In [1]:
import logging
import litellm

# litellm._turn_on_debug()
# # Configure logging
# logging.basicConfig(
#     level=logging.DEBUG,
#     format='%(asctime)s [%(levelname)s] %(name)s: %(message)s'
# )

# Optionally, set LiteLLM's internal logging to DEBUG
# litellm.set_verbose(True)

In [2]:
import os
os.chdir("SkyRL")

In [3]:
from gepa_artifact.gepa_artifact.benchmarks.hover import benchmark as hover_metas
from gepa_artifact.gepa_artifact.benchmarks.hover.hover_program import search
from functools import partial
retrieve_k = partial(search, k=7)
retrieve_10 = partial(search, k=10)

In [4]:
hoverbench = hover_metas[0].benchmark()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18171/18171 [00:01<00:00, 13329.70it/s]


In [5]:
for dset in [hoverbench.train_set, hoverbench.val_set, hoverbench.test_set]:
    for ex in dset:
        retrieved_docs_hop1 = retrieve_k(ex['claim'])['passages']
        ex['hop1_retrieved_docs'] = retrieved_docs_hop1

In [6]:
len(hoverbench.train_set), len(hoverbench.val_set), len(hoverbench.test_set)

(150, 300, 300)

In [7]:
for k, v  in hoverbench.train_set[1].items():
    print(k)

claim
supporting_facts
label
hop1_retrieved_docs


In [8]:
retrieve_k("Hi")['passages']

['Hi Hi (Puffy AmiYumi) | "Hi Hi" (not to be confused with the compilation album Hi Hi Puffy AmiYumi) is a single by Puffy AmiYumi released on 16 November 2005.  It is the theme song for the 2004 Cartoon Network animated series, "Hi Hi Puffy AmiYumi".',
 'Hi Hi Puffy Bu | "Hi Hi Puffy Club", more accurately titled "Hi Hi Puffy Bu" (Hi Hi Puffy 部 "Hi Hi Puffy Kurabu") is Puffy AmiYumi\'s reality television series, which premiered on TV Asahi on 22 July 2007.  The show stars both Ami Onuki and Yumi Yoshimura, and its opening credits include them in a cel-shaded manga.  The theme song for the series is "Shall We Dance?"  from their "Splurge" album.',
 'Hi, Hi, Hi | "Hi, Hi, Hi" is a song written by Paul and Linda McCartney and performed by Wings.  It was released as a double A-side single with "C Moon" in 1972.  The song was recorded around the same time as "C Moon", in November 1972.',
 'Hi Hi Puffy AmiYumi: The Genie and the Amp | Hi Hi Puffy AmiYumi: The Genie & the Amp is an 2006 acti

In [9]:
def format_user_message_for_hover(datapoint):
    retrieved_docs_hop1 = retrieve_k(datapoint['claim'])['passages']
    return f"""Claim: {datapoint['claim']}
Documents: {str(retrieved_docs_hop1)}
"""

def create_messages(datapoint, system_prompt="Given a claim, and some documents related to the claim, write a query to retrieve documents supporting the claim. Respond with your reasoning and the query at the end formatted as '### reasoning: <your reasoning here>\n### query: <your query here>'"):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": format_user_message_for_hover(datapoint)
        }
    ]
    return messages

In [10]:
messages = create_messages(hoverbench.train_set[1])

In [11]:
import litellm

In [12]:
response = litellm.completion(
    model="hosted_vllm/Qwen/Qwen3-8B",                   # For example: "hosted_vllm/meta-llama/Meta-Llama-3-8B-Instruct"
    messages=messages,
    api_base="http://localhost:8000/v1"                 # vLLM's OpenAI-compatible endpoint
)

InternalServerError: litellm.InternalServerError: InternalServerError: Hosted_vllmException - Connection error.

In [ ]:
response

ModelResponse(id='chatcmpl-e694bbf9315945579898590a1f744374', created=1758726915, model='hosted_vllm/Qwen/Qwen3-8B', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='<think>\nOkay, let\'s tackle this problem. The user wants me to create a query that retrieves documents supporting the claim that "Handsome Harry was the first project produced by an Soviet independent motion picture company that produced a 2015 mystery thriller film based on a novel by Tom Rob." \n\nFirst, I need to break down the claim into parts. The main points are:\n\n1. Handsome Harry is the first project of a Soviet independent film company.\n2. The company produced a 2015 mystery thriller film.\n3. That film is based on a novel by Tom Rob.\n\nLooking at the provided documents, let\'s check each one.\n\nThe first document is about "Child 44," a 2015 film based on Tom Rob Smith\'s novel. It mentions the director, writer, and cast. However, it 

In [ ]:
print(response.choices[0].message.content)

<think>
Okay, let's tackle this problem. The user wants me to create a query that retrieves documents supporting the claim that "Handsome Harry was the first project produced by an Soviet independent motion picture company that produced a 2015 mystery thriller film based on a novel by Tom Rob." 

First, I need to break down the claim into parts. The main points are:

1. Handsome Harry is the first project of a Soviet independent film company.
2. The company produced a 2015 mystery thriller film.
3. That film is based on a novel by Tom Rob.

Looking at the provided documents, let's check each one.

The first document is about "Child 44," a 2015 film based on Tom Rob Smith's novel. It mentions the director, writer, and cast. However, it doesn't mention anything about Handsome Harry or Soviet companies. So this document supports part of the claim (the 2015 film based on Tom Rob's novel) but not the connection to Handsome Harry or Soviet companies.

The second document is about "Handsome H

In [13]:
from skyrl_gym.envs.hover.env import HoVerEnv, DEFAULT_SYSTEM_PROMPT

In [14]:
env = HoVerEnv(None, {'reward_model': {'method': 'rule', **hoverbench.train_set[1]}})

In [15]:
env

In [16]:
o = env.step(response.choices[0].message.content)

NameError: name 'response' is not defined

In [17]:
o

NameError: name 'o' is not defined

In [18]:
import argparse
import re
import os
import random
import pandas as pd
import datasets

output_dir = "/home/ubuntu/SkyRL/hover_data"
os.makedirs(output_dir, exist_ok=True)

data_source = "hover"

from  vllm.transformers_utils import tokenizer
tokenizer = tokenizer.get_tokenizer(tokenizer_name="Qwen/Qwen3-8B")

max_tokens_in_prompt = 0

# add a row to each data item that represents a unique id
def process_fn(example, idx, split_name):
    global max_tokens_in_prompt
    print(example, idx)
    data = {
        "data_source": data_source + "_" + split_name,
        "prompt": create_messages(example, DEFAULT_SYSTEM_PROMPT),
        "env_class": "hover",
        "reward_model": {
            "method": "rule",
            "dset_idx": idx,
            **example,
        },
        "id": idx
    }
    prompt_len = len(tokenizer.apply_chat_template(data['prompt']))
    max_tokens_in_prompt = max(prompt_len, max_tokens_in_prompt)
    return data

for split_name, dset in [
    ("train", hoverbench.train_set),
    ("validation", hoverbench.val_set),
    ("test", hoverbench.test_set)
]:
    parquet_dset = [process_fn(d, idx, split_name) for idx, d in enumerate(dset)]
    parquet_dset = pd.DataFrame(parquet_dset)
    parquet_dset.to_parquet(os.path.join(output_dir, f"{split_name}.parquet"))

# train_dataset = train_dataset.map(function=make_map_fn("train"), with_indices=True)
# train_split = list(range(len(train_dataset)))
# random.Random(0).shuffle(train_split)
# train_dataset = train_dataset.select(train_split)
# first_half_count = 45
# second_half_count = 45
# num_examples = len(train_dataset)
# first_half = train_dataset.select(range(first_half_count))
# second_half = train_dataset.select(range(num_examples - second_half_count, num_examples))
# repeated_times = 23
# train_repeated = datasets.concatenate_datasets([first_half] * repeated_times)
# # val_dataset = val_dataset.map(function=make_map_fn("test"), with_indices=True)

# output_dir = args.output_dir
# os.makedirs(output_dir, exist_ok=True)
# print(f"Training dataset size: {len(train_repeated)}")
# print(f"Validation dataset size: {len(second_half)}")
# train_repeated.to_parquet(os.path.join(output_dir, "train.parquet"))
# second_half.to_parquet(os.path.join(output_dir, "validation.parquet"))
# # val_dataset.to_parquet(os.path.join(output_dir, "validation.parquet"))

Example({'claim': 'The city containing the Parafield railway station is not very near the airport and the Mawson Lakes campus of the University of South Australia.', 'supporting_facts': [{'key': 'Parafield, South Australia', 'value': 1}, {'key': 'Parafield Airport', 'value': 0}, {'key': 'Parafield railway station', 'value': 1}], 'label': 0, 'hop1_retrieved_docs': ["Parafield Airport | Parafield Airport (ICAO: YPPF) is on the edge of the residential suburb of Parafield, South Australia, 18 km north of the Adelaide city centre and adjacent to the Mawson Lakes campus of the University of South Australia.  It is Adelaide's second airport and the fifth busiest airport in Australia by aircraft movements.  Although owned by the Government of Australia, the airport is leased to and managed independently by Parafield Airport Ltd., a wholly owned subsidiary of Adelaide Airport Limited.", 'Mawson Lakes, South Australia | Mawson Lakes is a residential suburb in the City of Salisbury, Adelaide, Aus

In [19]:
hover_train_pqt = pd.read_parquet("/home/ubuntu/SkyRL/hover_data/train.parquet")
aime_train_pqt = pd.read_parquet("/home/ubuntu/SkyRL/aime/validation.parquet")

In [20]:
hover_train_pqt.iloc[1]

data_source                                           hover_train
prompt          [{'content': 'Given a claim, and some document...
env_class                                                   hover
reward_model    {'claim': 'Handsome Harry was the first projec...
id                                                              1
Name: 1, dtype: object

In [21]:
aime_train_pqt.iloc[1]

id                                                             25
solution        Let $a$, $b$, and $c$ be the vertex of a trian...
url             https://artofproblemsolving.com/wiki/index.php...
data_source                            AI-MO/aimo-validation-aime
prompt          [{'content': 'Twenty distinct points are marke...
env_class                                                    aime
reward_model            {'ground_truth': '072', 'method': 'rule'}
Name: 1, dtype: object

In [22]:
import tiktoken

In [23]:
max_tokens_in_prompt

2092

In [24]:
hover_train_pqt.shape

(150, 5)

In [25]:
aime_train_pqt.shape

(45, 7)

In [26]:
1035/150

6.9

In [ ]:
2*6.9

13.8

In [27]:
import pickle

In [28]:
with open("/home/ubuntu/SkyRL/gepa_hover_300_7000/gepa_state.bin", "rb") as fb:
    obj = pickle.load(fb)

In [29]:
obj

{'program_candidates': [{'instruction_prompt': "Given a claim, and some documents related to the claim, write a query to retrieve documents supporting the claim. Respond with your reasoning and the query at the end formatted as '### reasoning: <your reasoning here>\n### query: <your query here>'"},
  {'instruction_prompt': 'Given a claim and a set of documents, your task is to generate a query that retrieves documents supporting the claim. Follow these steps:  \n\n1. **Analyze the claim** to identify key entities, relationships, and specific details (e.g., names, locations, roles, or attributes).  \n2. **Examine the documents** to determine:  \n   - Whether the claim is explicitly supported, contradicted, or partially addressed.  \n   - Which documents contain relevant information (e.g., direct mentions, indirect references, or contextual clues).  \n3. **Formulate a query** that:  \n   - Uses precise terms from the claim (e.g., exact names, titles, or locations).  \n   - Includes synon

In [32]:
obj['program_full_scores_test_set']

[0.3,
 0.38333333333333336,
 0.41333333333333333,
 0.38,
 0.44,
 0.39666666666666667,
 0.44,
 0.38666666666666666,
 0.44333333333333336,
 0.42333333333333334,
 0.39666666666666667,
 0.4033333333333333,
 0.4166666666666667,
 0.4,
 0.39666666666666667,
 0.4166666666666667,
 0.3933333333333333,
 0.39,
 0.38,
 0.46,
 0.39666666666666667,
 0.38666666666666666]